# Assignment 4: Permutation test 1
## Learning Outcomes
By completing this assignment, you will be able to implement and use randomization tests to determine whether a specific effect is statistically significant. Randomization tests are a highly universal and powerful way to do this, and form an essential part of the toolkit of any Data Scientist. 

## Problem Description
An increasingly common statistical tool for determining whether a difference is significant is the randomization or permutation test. A randomization test builds - rather than assumes – a sampling distribution of the test statistics. This is achieved by exchanging or permuting variables which are “exchangeable” under the Null-hypothesis. A randomization test gives a simple way to compute the sampling distribution for any test statistic under the null hypothesis that the dependent variable is statistically not dependent on the shuffled variable. 
In this assignment, we continue with the movie dataset from last week. This week your main task is to determine whether the difference in proportion for female vs. male directors to direct action movies is significant. 
You are allowed to use standard numpy and pandas functions such as mean, groupby, shuffle. But you are not allowed to use a permutation functions that have been already been designed. Numeric calculations in python that are printed should be rounded to 6 decimal places. 

Methods that you may find beneficial. 
Pandas: crosstab, iloc. Numpy: random.shuffle, reset_index, matplotlib: hist, axvline 

## Preliminaries 
Import pandas, numpy, matplotlib, and load the dataset file

In [355]:
# importing packages
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
%matplotlib inline

import numpy as np
import pandas as pd
# import scipy.stats as stats

In [356]:
df = pd.read_csv("movieDataset.csv")

## TASK 1: Defining the test statistics (40pts)
### Question 1 - Create contingency table of adventure (yes/no) by director gender (10pts)
Generate a cross-tabulation table of the 2x2 proportions of directorGender vs. whether the movie is an adventure or not (all other genres should be combined into non-adventure movies).

In [357]:
ctable = pd.crosstab(df['genre'] == 'adventure', df['dirGender'], margins = True)
display(ctable)

dirGender,female,male,All
genre,,,
False,31,155,186
True,6,58,64
All,37,213,250


### Question 2 - Calculate p(adventure|female)-p(adventure|male) (20pts)
Calculate the probability that the movie is an adventure given that the director Gender is female, p(adventure|female), the probability of the movie is an adventure given that the director Gender is male, p(adventure|male), as well as the difference between these two numbers, p(adventure|female)- p(adventure|male) (for help, see solution to homework 2).

In [358]:
p_female = ctable['female'][True] / ctable['female']['All']
print("the probability that the movie is an adventure given that the director is female: %.6f" % (p_female))
p_male = ctable['male'][True] / ctable['male']['All']
print("the probability that the movie is an adventure given that the director is male: %.6f" % (p_male))
p_diff = p_female - p_male
print("the difference between the probability these probabilities are %.6f" % (p_diff))


the probability that the movie is an adventure given that the director is female: 0.162162
the probability that the movie is an adventure given that the director is male: 0.272300
the difference between the probability these probabilities are -0.110138


### Question 3 - Define a function that calculates the difference in average income by gender (10pts)
Write a function (e.g., https://www.tutorialspoint.com/python/python_functions.htm) that takes the data frame as an input and returns the difference between the average income of male directors in adventure and the average income of female directors in adventure.

In [359]:
def avg_inc(dataframe, col_name = "dirIncome"):
    """
        
        we want:
        male : average dirIncome for adventure genre
        female : average dirIncome for adventure genre
        
    """
    
    grouped_gend = dataframe.groupby(df['dirGender'])
    male_g = grouped_gend.get_group('male')
    female_g = grouped_gend.get_group('female')
    male_adv = male_g.groupby(df['genre'])
    female_adv = female_g.groupby(df['genre'])
    male = male_adv.get_group('adventure')
    female = female_adv.get_group('adventure')
    col_mean_m = male[col_name].mean()
    col_mean_f = female[col_name].mean()

    
    
    #print("the average male director income: %.6f" % (col_mean_m))
    #print("the average female director income: %.6f" % (col_mean_f))
    
    diff = col_mean_m - col_mean_f

    
    return diff

In [360]:
test = avg_inc(df)
print(test)

1.44593386942614


## Task 2: Perform a permutation test (60pts)
### Question 1 (10pts)
Written answer: We want to test the hypothesis that female directors of adventure movies earn less, on average, than male directors of adventure movies. What is the Null-hypothesis we need to consider? 

If we want to test the hypothesis that female directors of adventure movies earn less on average than male directors, our null-hypothesis would be that female directors earn the same if not more on average as male directors of adventure movies.

### Question 2 (40pts)
Perform a permutation test. Under the Null-hypothesis the director gender is exchangeable. Write a function that takes the data frame as an input and then randomly permutes the directorGender column. For each iteration, the function then calls the function written for Task1, Q3 to get the test statistic. Each iteration, the test statistic should  be stored in a list. It is important to mention that the gender must be randomly assigned each time prior to calculating the difference of the conditional probabilities. 

After bulding up the numpy array of test statistics, the function should plot a histogram of the test statistics and mark the value of the empirical test statistics by a vertical line (see https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.axvline.html). Finally, it should report the p-value. 

For Pseudo code see lecture. Start with 500 iterations to test your code – for the final result, use 5000 iterations.

p=randomizationTest(Data)
      calculate test statistics from Data -> y
      for numIter times
            reassign the independent
               variable randomly -> Data*
            calculate test statistics from Data*
            record test statistics in array X
      plot histogram of X
      see where y falls in X
      p = the proportion of X >= y

In [384]:
def randomizationTest(dataframe, col_name = "dirGender"):
    real_stats = avg_inc(dataframe)
    df_copy = dataframe.copy()
    n_rows = len(df_copy.index)
    #print(n_rows)
    indices = np.arange(n_rows)
    #print(indices)
    np.random.shuffle(indices)
    #print(indices)
    array = np.empty(500,dtype=object)
    for numIter in range(500):
        shuffle_gender = df_copy["dirGender"].iloc[indices]
        #print(shuffle_gender)
        #print(df["dirGender"])
        shuffle_gender = shuffle_gender.reset_index(drop = True)
        #print(shuffle_gender)
        df_copy["dirGender"] = shuffle_gender
        
        #ctab3 = pd.crosstab(df["dirGender"], df["genre"], margins = True)
        #ctab4 = pd.crosstab(df_copy["dirGender"], df_copy["genre"], margins = True)

        #display(ctab3)
        #display(ctab4)
        
        test_stats = avg_inc(df_copy)
        array[numIter] = test_stats
    return array
    

In [385]:
random = randomizationTest(df)
print(random)
#real = avg_inc(df)
#print(real)

[1.44593386942614 1.44593386942614 1.44593386942614 1.44593386942614
 1.44593386942614 1.44593386942614 1.44593386942614 1.44593386942614
 1.44593386942614 1.44593386942614 1.44593386942614 1.44593386942614
 1.44593386942614 1.44593386942614 1.44593386942614 1.44593386942614
 1.44593386942614 1.44593386942614 1.44593386942614 1.44593386942614
 1.44593386942614 1.44593386942614 1.44593386942614 1.44593386942614
 1.44593386942614 1.44593386942614 1.44593386942614 1.44593386942614
 1.44593386942614 1.44593386942614 1.44593386942614 1.44593386942614
 1.44593386942614 1.44593386942614 1.44593386942614 1.44593386942614
 1.44593386942614 1.44593386942614 1.44593386942614 1.44593386942614
 1.44593386942614 1.44593386942614 1.44593386942614 1.44593386942614
 1.44593386942614 1.44593386942614 1.44593386942614 1.44593386942614
 1.44593386942614 1.44593386942614 1.44593386942614 1.44593386942614
 1.44593386942614 1.44593386942614 1.44593386942614 1.44593386942614
 1.44593386942614 1.44593386942614

### Question 3 (10pts)
Written response: What do you conclude from this result?
